# Autoregressive 1D CNN (Conv1d)

These notebooks implement the Chapter 18 CNN models (time series as images) on this repo's dataset.

Common requirements satisfied in each notebook:
- **Time-wise split**: train = first 7 years, val = middle, test = last 18 months
- Uses the existing backtester: `src/backtester.engine.run_backtest`
- Uses the existing Bokeh dashboard: `src/backtester.bokeh_plots.build_interactive_portfolio_layout`
- Backtest window is sliced to the test period so reported Start/End align with the test start (not 2016).

GPU note (AMD): PyTorch uses `torch.cuda` for both CUDA and ROCm builds. If your PyTorch build supports ROCm, `torch.cuda.is_available()` should be True.


## Math: Convolution and Pooling

For a 1D signal x and kernel w, a discrete 1D convolution (cross-correlation in many DL libs) is:

$$y[t] = \sum_{k=0}^{K-1} w[k] \cdot x[t+k].$$

For multichannel inputs, the kernel has shape (C_in, K), and outputs sum over channels.

Pooling reduces resolution while keeping salient local patterns. For example, max-pooling over window size P:

$$y[t] = \max_{0\le k < P} x[t+k].$$

CNNs exploit locality and weight sharing to reduce parameters vs fully-connected layers.


## Data, Label, and Portfolio Construction

We load the repo's feature-extracted panel dataset and create a forward label per asset:

$$y_t = r_{t+1}$$

where `ret_1d` is the known one-day return at time t.

Binary direction label:

$$y^{\uparrow}_t = \mathbb{1}[y_t > 0].$$

From a model probability $p_{i,t}$, define a centered score $s_{i,t} = p_{i,t} - 0.5$.

Weekly Top-K long-only weights at rebalance date t:

$$w_{i,t}=\frac{1}{|\mathcal{L}_t|}\mathbb{1}[i\in\mathcal{L}_t],\quad \mathcal{L}_t=\{\text{TopK}(s_{\cdot,t})\cap (s_{\cdot,t}>0)\}.$$


## Autoregressive 1D CNN (Time Series)

We use a 1D convolution over time.
Input tensor per sample: (C, L) where:
- C = number of channels (selected features)
- L = lookback window length (days)

A Conv1D layer learns local temporal filters shared across time.


In [1]:
from __future__ import annotations

from pathlib import Path
import sys

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from bokeh.io import output_notebook, show


In [2]:
# Resolve project root robustly by walking parents.
CWD = Path.cwd().resolve()
PROJECT_ROOT = None
for p in [CWD, *CWD.parents]:
    if (p / 'dataset').exists() and (p / 'src').exists():
        PROJECT_ROOT = p
        break
if PROJECT_ROOT is None:
    raise RuntimeError(f'Could not locate project root from CWD={CWD}')
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
print('PROJECT_ROOT:', PROJECT_ROOT)


PROJECT_ROOT: /home/anivarth/college/quant-task


In [3]:
# Device selection: CUDA on NVIDIA, ROCm on AMD (if your PyTorch build supports it).
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('torch version:', torch.__version__)
print('device:', device)


torch version: 2.10.0+cpu
device: cpu


In [4]:
SEED = 42
rng = np.random.default_rng(SEED)
torch.manual_seed(SEED)

FEATURES_PARQUET_PATH = PROJECT_ROOT / 'dataset' / 'features' / 'all_features.parquet'
FEATURES_CSV_PATH = PROJECT_ROOT / 'dataset' / 'features' / 'all_features.csv'

TARGET_COL = 'ret_1d'
TARGET_FWD_COL = 'y_ret_1d_fwd'
TARGET_DIR_COL = 'y_up_fwd'

TRAIN_YEARS = 7
TEST_MONTHS = 18

REBALANCE_FREQ = 'W'
TOP_K = 20
INITIAL_EQUITY = 1_000_000.0
TXN_COST_BPS = 5.0


def load_feature_dataset() -> pd.DataFrame:
    if FEATURES_PARQUET_PATH.exists():
        df = pd.read_parquet(FEATURES_PARQUET_PATH)
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.set_index('Date')
    elif FEATURES_CSV_PATH.exists():
        df = pd.read_csv(FEATURES_CSV_PATH, parse_dates=['Date']).set_index('Date')
    else:
        raise FileNotFoundError('Feature dataset not found under dataset/features/.')

    required = {'Asset_ID', TARGET_COL}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f'Missing required columns: {sorted(missing)}')

    return df.sort_index()

panel = load_feature_dataset().copy()

panel[TARGET_FWD_COL] = panel.groupby('Asset_ID', sort=False)[TARGET_COL].shift(-1)
panel = panel.dropna(subset=[TARGET_FWD_COL]).sort_index()
panel[TARGET_DIR_COL] = (panel[TARGET_FWD_COL].astype(float) > 0.0).astype(int)

start = pd.Timestamp(panel.index.min())
end = pd.Timestamp(panel.index.max())
train_end = start + pd.DateOffset(years=TRAIN_YEARS)
test_start = end - pd.DateOffset(months=TEST_MONTHS)

if train_end >= test_start:
    raise ValueError(
        f'Not enough history for requested split: start={start.date()} train_end={train_end.date()} test_start={test_start.date()} end={end.date()}'
    )

train_mask = panel.index < train_end
val_mask = (panel.index >= train_end) & (panel.index < test_start)
test_mask = panel.index >= test_start

panel_train = panel.loc[train_mask].copy()
panel_val = panel.loc[val_mask].copy()
panel_test = panel.loc[test_mask].copy()

print('date range:', start.date(), '->', end.date())
print('train:', panel_train.index.min().date(), '->', panel_train.index.max().date(), 'rows:', panel_train.shape[0])
print('val  :', panel_val.index.min().date(), '->', panel_val.index.max().date(), 'rows:', panel_val.shape[0])
print('test :', panel_test.index.min().date(), '->', panel_test.index.max().date(), 'rows:', panel_test.shape[0])
print('assets:', panel['Asset_ID'].nunique())

exclude = {'Asset_ID', TARGET_FWD_COL, TARGET_DIR_COL}
feature_cols = [c for c in panel.columns if c not in exclude]
numeric_feature_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(panel[c])]
print('n_numeric_features:', len(numeric_feature_cols))


date range: 2016-01-25 -> 2026-01-15
train: 2016-01-25 -> 2023-01-24 rows: 176300
val  : 2023-01-25 -> 2024-07-12 rows: 36800
test : 2024-07-15 -> 2026-01-15 rows: 37900
assets: 100
n_numeric_features: 127


In [5]:
from sklearn.feature_selection import mutual_info_classif

LOOKBACK = 60
N_CHANNELS = 16
BATCH_SIZE = 512
EPOCHS = 3
LR = 1e-3

# Feature selection on training rows (per-date panel, no lags yet)
X_fs = panel_train[numeric_feature_cols].replace([np.inf, -np.inf], np.nan)
y_fs = panel_train[TARGET_DIR_COL].astype(int)

# Median impute for MI scoring
med = X_fs.median(axis=0)
X_fs_imp = X_fs.fillna(med)

mi = mutual_info_classif(X_fs_imp.to_numpy(), y_fs.to_numpy(), random_state=SEED)
mi_s = pd.Series(mi, index=numeric_feature_cols).sort_values(ascending=False)
sel_features = mi_s.head(N_CHANNELS).index.tolist()
print('selected features:', sel_features)

# Build per-asset sequences
panel_use = panel.loc[:, ['Asset_ID', TARGET_DIR_COL] + sel_features].copy()
panel_use = panel_use.replace([np.inf, -np.inf], np.nan)

# Train statistics for normalization (computed on train window only)
train_vals = panel_train.loc[:, sel_features].replace([np.inf, -np.inf], np.nan)
train_med = train_vals.median(axis=0)
train_mean = train_vals.fillna(train_med).mean(axis=0)
train_std = train_vals.fillna(train_med).std(axis=0, ddof=0).replace(0.0, 1.0)


def norm_frame(df: pd.DataFrame) -> pd.DataFrame:
    x = df.loc[:, sel_features].fillna(train_med)
    x = (x - train_mean) / train_std
    return x


def build_sequences(df: pd.DataFrame):
    # returns arrays: X (n, C, L), y (n,), dates (n,), assets (n,)
    X_list = []
    y_list = []
    d_list = []
    a_list = []

    for asset_id, g in df.groupby('Asset_ID', sort=False):
        g = g.sort_index()
        x = norm_frame(g)
        y = g[TARGET_DIR_COL].astype(int)
        dates = g.index

        x_np = x.to_numpy(dtype=np.float32)
        y_np = y.to_numpy(dtype=np.int64)

        for i in range(LOOKBACK - 1, len(g)):
            # sample date is dates[i], uses lookback ending at i
            Xw = x_np[i - LOOKBACK + 1 : i + 1, :].T  # (C, L)
            X_list.append(Xw)
            y_list.append(y_np[i])
            d_list.append(pd.Timestamp(dates[i]))
            a_list.append(asset_id)

    X = np.stack(X_list, axis=0) if X_list else np.zeros((0, N_CHANNELS, LOOKBACK), dtype=np.float32)
    y = np.array(y_list, dtype=np.int64)
    return X, y, np.array(d_list, dtype='datetime64[ns]'), np.array(a_list)

# Build splits based on the *sample date*
X_tr, y_tr, d_tr, a_tr = build_sequences(panel_use.loc[train_mask])
X_va, y_va, d_va, a_va = build_sequences(panel_use.loc[val_mask])
X_te, y_te, d_te, a_te = build_sequences(panel_use.loc[test_mask])

print('X_tr:', X_tr.shape, 'y_tr:', y_tr.shape)
print('X_va:', X_va.shape, 'y_va:', y_va.shape)
print('X_te:', X_te.shape, 'y_te:', y_te.shape)

class SeqDS(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_loader = DataLoader(SeqDS(X_tr, y_tr), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(SeqDS(X_va, y_va), batch_size=BATCH_SIZE, shuffle=False)

class ARConv1D(nn.Module):
    def __init__(self, c_in: int, lookback: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(c_in, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(64, 1),
        )
    def forward(self, x):
        return self.net(x).squeeze(-1)

model = ARConv1D(N_CHANNELS, LOOKBACK).to(device)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.BCEWithLogitsLoss()

for epoch in range(1, EPOCHS + 1):
    model.train()
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.float().to(device)
        opt.zero_grad(set_to_none=True)
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        opt.step()

    model.eval()
    # simple val AUC (subsample to keep runtime bounded)
    all_p = []
    all_y = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            logits = model(xb)
            p = torch.sigmoid(logits).cpu().numpy()
            all_p.append(p)
            all_y.append(yb.numpy())
    if all_p:
        import sklearn.metrics as skm
        p = np.concatenate(all_p)
        y = np.concatenate(all_y)
        if len(np.unique(y)) > 1:
            auc = float(skm.roc_auc_score(y, p))
            print('epoch', epoch, 'val_auc', auc)
        else:
            print('epoch', epoch, 'val_auc n/a')

# Predict on test
model.eval()
with torch.no_grad():
    logits = model(torch.from_numpy(X_te).to(device)).cpu().numpy()
proba = 1.0 / (1.0 + np.exp(-logits))

MODEL_TITLE = 'Autoregressive 1D CNN (Conv1d) - Time Split'

dates_test = pd.to_datetime(d_te)
assets_test = a_te
scores_test = proba - 0.5


selected features: ['diff_log_close_2', 'excess_ret_1d', 'macd_macd', 'logret_roll_std_20', 'diff_log_close_1', 'logret_1d', 'ret_1d', 'realized_vol_20', 'close_minmax_20', 'logret_roll_var_20', 'filt_close_wiener_5', 'close', 'macd_macd_hist', 'ret_lag_1', 'logret_lag_1', 'bos']
X_tr: (170400, 16, 60) y_tr: (170400,)
X_va: (30900, 16, 60) y_va: (30900,)
X_te: (32000, 16, 60) y_te: (32000,)
epoch 1 val_auc 0.4945845299541582
epoch 2 val_auc 0.49045954871808733
epoch 3 val_auc 0.4934425048493659


In [6]:
from src.backtester.data import load_cleaned_assets, align_close_prices
from src.backtester.engine import BacktestConfig, run_backtest
from src.backtester.report import compute_backtest_report
from src.backtester.bokeh_plots import build_interactive_portfolio_layout
from src.backtester.portfolio import equal_weight

# Convert per-row predictions to pred_matrix[date, asset]
long = pd.DataFrame({'Date': dates_test, 'Asset_ID': assets_test, 'score': scores_test})
pred_matrix = long.pivot_table(index='Date', columns='Asset_ID', values='score', aggfunc='mean').sort_index()

bt_assets = sorted(panel['Asset_ID'].unique().tolist())
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(PROJECT_ROOT / 'dataset' / 'cleaned'))
close_prices = align_close_prices(assets_ohlcv).sort_index()

# Slice backtest to TEST window (fixes Start not being 2016)
bt_start = pd.Timestamp(panel_test.index.min())
bt_end = pd.Timestamp(panel_test.index.max())
close_prices = close_prices.loc[bt_start:bt_end]

assets_ohlcv_slice = {k: v.loc[bt_start:bt_end] for k, v in assets_ohlcv.items()}
market_df = pd.DataFrame({
    'Open': pd.concat([d['Open'] for d in assets_ohlcv_slice.values()], axis=1).mean(axis=1),
    'High': pd.concat([d['High'] for d in assets_ohlcv_slice.values()], axis=1).mean(axis=1),
    'Low': pd.concat([d['Low'] for d in assets_ohlcv_slice.values()], axis=1).mean(axis=1),
    'Close': pd.concat([d['Close'] for d in assets_ohlcv_slice.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([d['Volume'] for d in assets_ohlcv_slice.values()], axis=1).sum(axis=1),
}).sort_index()

pred_matrix = pred_matrix.reindex(close_prices.index)
output_notebook()

rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())

w_last = pd.Series(0.0, index=bt_assets)
w_rows = []
for dt in pred_matrix.index:
    if dt in rebal_dates:
        row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
        top = row.head(min(TOP_K, len(row)))
        candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
        if len(candidates) == 0:
            w_last = pd.Series(0.0, index=bt_assets)
        else:
            w_dict = equal_weight(candidates)
            w_last = pd.Series(0.0, index=bt_assets)
            for a, w in w_dict.items():
                if a in w_last.index:
                    w_last[a] = float(w)
    w_rows.append(w_last)

weights = pd.DataFrame(w_rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=INITIAL_EQUITY, transaction_cost_bps=TXN_COST_BPS, mode='vectorized')
res = run_backtest(close_prices, weights, config=cfg)
report = compute_backtest_report(result=res, close_prices=close_prices)
display(report.to_frame(MODEL_TITLE))

layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close_prices,
    title=MODEL_TITLE,
)
show(layout)


Loading BokehJS ...

,Autoregressive 1D CNN (Conv1d) - Time Split
Start,2024-10-11 00:00:00
End,2026-01-15 00:00:00
Duration,461 days 00:00:00
Initial Equity,999500.0
Final Equity,1053899.627255
Equity Peak,1056166.329386
Total Return [%],5.442684
CAGR [%],4.33095
Volatility (ann) [%],18.502639
Sharpe,0.318121
